In [1]:
#!pip install splinter

In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Urls for Data scrape
freeThrowPct = "https://www.teamrankings.com/ncaa-basketball/stat/free-throw-pct"
astTo = "https://www.teamrankings.com/ncaa-basketball/stat/assist--per--turnover-ratio"
eFG = "https://www.teamrankings.com/ncaa-basketball/stat/effective-field-goal-pct"
defReb = "https://www.teamrankings.com/ncaa-basketball/stat/defensive-rebounding-pct"
offReb = "https://www.teamrankings.com/ncaa-basketball/stat/offensive-rebounding-pct"
oppEFG = "https://www.teamrankings.com/ncaa-basketball/stat/opponent-effective-field-goal-pct"
defEff = "https://www.teamrankings.com/ncaa-basketball/stat/defensive-efficiency"
teams = "https://www.teamrankings.com/ncaa-basketball/ranking/predictive-by-other"
sos = "https://www.teamrankings.com/ncaa-basketball/ranking/schedule-strength-by-other"

In [4]:
#Launch chrome driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jerkardash/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [5]:
#Process is iterative, visit browser
browser.visit(freeThrowPct)

In [6]:
#Create HTML object
html = browser.html

#Parse through with BS
soup = BeautifulSoup(html, 'html.parser')

In [7]:
#Get table from HTML
table = soup.find_all('table')
#Isolate fro DF
df = pd.read_html(str(table))[0]
df = df[["Team", "2020"]]

#Rename to catagory
df_ft = df.rename(columns={
    "2020": "Free Throw %"
})
#df_ft

In [8]:
browser.visit(astTo)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [9]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df = df[["Team", "2020"]]
df_ast = df.rename(columns={
    "2020": "AST/TO"
})
#df_ast

In [10]:
#Create a merged DF
merged = df_ft.merge(df_ast, on="Team")
#merged

In [11]:
browser.visit(eFG)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [12]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df = df[["Team", "2020"]]
df_eFG = df.rename(columns={
    "2020": "EFG %"
})
#df_eFG

In [13]:
merged = merged.merge(df_eFG, on="Team")
#merged

In [14]:
browser.visit(defReb)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df = df[["Team", "2020"]]
df_defReb = df.rename(columns={
    "2020": "Def Reb %"
})
#df_defReb

In [16]:
merged = merged.merge(df_defReb, on="Team")
#merged

In [17]:
browser.visit(offReb)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [18]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df = df[["Team", "2020"]]
df_offReb = df.rename(columns={
    "2020": "Off Reb %"
})
#df_offReb

In [19]:
merged = merged.merge(df_offReb, on="Team")
#merged

In [20]:
browser.visit(oppEFG)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df = df[["Team", "2020"]]
df_oppEFG = df.rename(columns={
    "2020": "Opp EFG %"
})
#df_oppEFG

In [22]:
merged = merged.merge(df_oppEFG, on="Team")
#merged

In [23]:
browser.visit(defEff)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [24]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df = df[["Team", "2020"]]
df_defEff = df.rename(columns={
    "2020": "Def Eff"
})
#df_defEff

In [25]:
merged = merged.merge(df_defEff, on="Team")
#merged

In [26]:
browser.visit(sos)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [27]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df = df[["Team", "Rating"]]
df_sos = df.rename(columns={
    "2020": "SOS"
})

In [28]:
#Take out record from team DF
df_sos["Team"] = df_sos["Team"].str.rsplit(" ", n=1, expand=True)

In [29]:
fatdood_data = merged.merge(df_sos, on="Team")
#fatdood_data

In [30]:
browser.quit()

In [31]:
df = fatdood_data

In [32]:
df.dtypes

Team             object
Free Throw %     object
AST/TO           object
EFG %            object
Def Reb %        object
Off Reb %        object
Opp EFG %        object
Def Eff          object
Rating          float64
dtype: object

In [33]:
df.loc[df["Free Throw %"] == "--", 'Free Throw %'] = np.nan
df = df.dropna()

In [34]:
#Take our percentage signs and make float to prepare for mean and SD

df["Free Throw %"] = df["Free Throw %"].str[:-1]
df["Free Throw %"] = df["Free Throw %"].astype(float)

df["AST/TO"] = df["AST/TO"].astype(float)

df["EFG %"] = df["EFG %"].str[:-1]
df["EFG %"] = df["EFG %"].astype(float)

df["Def Reb %"] = df["Def Reb %"].str[:-1]
df["Def Reb %"] = df["Def Reb %"].astype(float)

df["Off Reb %"] = df["Off Reb %"].str[:-1]
df["Off Reb %"] = df["Off Reb %"].astype(float)

df["Opp EFG %"] = df["Opp EFG %"].str[:-1]
df["Opp EFG %"] = df["Opp EFG %"].astype(float)

df["Def Eff"] = df["Def Eff"].astype(float)

In [35]:
#Get mean of columns
ft_mean = df["Free Throw %"].mean()
ast_mean = df["AST/TO"].mean()
efg_mean = df["EFG %"].mean()
defReb_mean = df["Def Reb %"].mean()
offReb_mean = df["Off Reb %"].mean()
oefg_mean = df["Opp EFG %"].mean()
defEff_mean = df["Def Eff"].mean()
sos_mean = df["Rating"].mean()

In [36]:
#Get SD of columns
ft_sd = df["Free Throw %"].std()
ast_sd = df["AST/TO"].std()
efg_sd = df["EFG %"].std()
defReb_sd = df["Def Reb %"].std()
offReb_sd = df["Off Reb %"].std()
oefg_sd = df["Opp EFG %"].std()
defEff_sd = df["Def Eff"].std()
sos_sd = df["Rating"].std()

In [37]:
#Create Z-score for each value
df["FT Score"] = (df["Free Throw %"] - ft_mean)/ft_sd
df["AST Score"] = (df["AST/TO"] - ast_mean)/ast_sd
df["EFG Score"] = (df["EFG %"] - efg_mean)/efg_sd
df["Def Reb Score"] = (df["Def Reb %"] - defReb_mean)/defReb_sd
df["Off Reb Score"] = (df["Off Reb %"] - offReb_mean)/offReb_sd
df["Opp EFG Score"] = (df["Opp EFG %"] - oefg_mean)/oefg_sd
df["Def Eff Score"] = (df["Def Eff"] - defEff_mean)/defEff_sd
df["Rating Score"] = (df["Rating"] - sos_mean)/sos_sd

In [38]:
#Create overall score
df["Score"] = (df["FT Score"] + df["AST Score"] + df["EFG Score"] + df["Def Reb Score"] + df["Off Reb Score"] - df["Opp EFG Score"] - df["Def Eff Score"] + df["Rating Score"])/8 

In [39]:
#Sort by average Z-score
df = df.sort_values(by="Score", ascending=False)
df

,Team,Free Throw %,AST/TO,EFG %,Def Reb %,Off Reb %,Opp EFG %,Def Eff,Rating,FT Score,AST Score,EFG Score,Def Reb Score,Off Reb Score,Opp EFG Score,Def Eff Score,Rating Score,Score
111,Gonzaga,72.7,1.511,61.0,79.1,28.3,47.5,0.901,10.0,0.421887,2.394806,3.542185,1.526557,0.546350,-0.939524,-1.697134,1.567275,1.579465
68,Houston,74.1,1.212,50.8,76.8,39.6,43.4,0.860,6.3,0.755523,0.990856,0.260107,0.852470,3.026715,-2.360445,-2.432369,0.995588,1.459259
20,Michigan,77.8,1.395,54.9,78.4,27.6,44.9,0.942,12.4,1.637276,1.850129,1.579374,1.321400,0.392699,-1.840596,-0.961899,1.938099,1.440184
2,Virginia,81.7,1.482,56.1,80.3,21.1,47.9,0.969,10.2,2.566691,2.258637,1.965501,1.878255,-1.034060,-0.800897,-0.477719,1.598177,1.313977
245,Illinois,68.8,1.231,55.8,79.9,31.1,46.4,0.932,13.7,-0.507528,1.080070,1.868969,1.761022,1.160953,-1.320747,-1.141224,2.138962,1.245552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Air Force,69.3,0.853,52.1,65.6,13.3,57.4,1.107,-1.0,-0.388372,-0.694824,0.678411,-2.430042,-2.746170,2.491481,1.996975,-0.132335,-1.275223
316,San Jose St,65.6,0.755,44.7,66.1,18.2,52.6,1.127,-5.0,-1.270125,-1.154981,-1.702704,-2.283501,-1.670614,0.827964,2.355626,-0.750375,-1.501986
338,S Car State,62.1,0.571,44.8,66.8,24.1,55.4,1.065,-7.3,-2.104216,-2.018950,-1.670527,-2.078344,-0.375556,1.798349,1.243807,-1.105748,-1.549437
345,Miss Val St,57.2,0.614,43.6,64.7,15.8,54.9,1.161,-12.4,-3.271942,-1.817044,-2.056654,-2.693815,-2.197417,1.625066,2.965333,-1.893749,-2.315128


In [40]:
#Export to excel
df.to_excel("fatdood.xlsx", sheet_name='All Teams')